<a href="https://colab.research.google.com/github/santhapriyan17/CAS03-CS03/blob/main/CASTILO_Hackathon_team_Imposters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Enhanced DDoS Detection
!pip install scapy seaborn tensorflow scikit-learn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from collections import deque, defaultdict
import ipaddress
import time
import random
from scipy.stats import entropy

# Constants
SIMULATION_DURATION = 300
PACKET_INTERVAL = 0.001
NUM_BOTNET_IPS = 100
WINDOW_SIZE = 500

class NetworkPacket:
    def __init__(self, src_ip, dst_ip, src_port, dst_port, protocol, size, flags=None):
        self.src_ip = src_ip
        self.dst_ip = dst_ip
        self.src_port = src_port
        self.dst_port = dst_port
        self.protocol = protocol
        self.size = size
        self.flags = flags or {}
        self.timestamp = time.time()

class DDoSDetector:
    def __init__(self):
        self.feature_names = [
            'src_ip', 'dst_ip', 'src_port', 'dst_port', 'protocol',
            'size', 'flags', 'entropy_src', 'entropy_dst', 'packet_rate'
        ]

        self.models = {
            'rf': RandomForestClassifier(n_estimators=150),
            'iso': IsolationForest(contamination=0.15),
            'autoencoder': self.build_autoencoder()
        }

        self.scaler = StandardScaler()
        self.history = deque(maxlen=WINDOW_SIZE)
        self.botnet_ips = [f"10.0.0.{i}" for i in range(1, NUM_BOTNET_IPS+1)]

        self.metrics = {
            'timestamps': [], 'entropy_src': [], 'entropy_dst': [],
            'packet_rates': [], 'sizes': [], 'true_labels': [],
            'predictions': [], 'actions': [], 'anomaly_scores': [],
            'blocked_ips': set(), 'attack_types': defaultdict(int)
        }

        self.attack_generators = {
            'http2_rapid_reset': self._gen_http2_reset,
            'syn_flood': self._gen_syn_flood,
            'udp_flood': self._gen_udp_flood,
            'dns_amplification': self._gen_dns_amp,
            'slowloris': self._gen_slowloris,
            'ssl_flood': self._gen_ssl_flood
        }

    def build_autoencoder(self):
        input_layer = Input(shape=(len(self.feature_names),))
        encoded = Dense(32, activation='relu')(input_layer)
        encoded = Dense(16, activation='relu')(encoded)
        decoded = Dense(32, activation='relu')(encoded)
        decoded = Dense(len(self.feature_names), activation='linear')(decoded)
        autoencoder = Model(input_layer, decoded)
        autoencoder.compile(optimizer='adam', loss='mse')
        return autoencoder

    # Attack generation methods
    def _gen_http2_reset(self):
        return NetworkPacket(
            src_ip=random.choice(self.botnet_ips),
            dst_ip="192.168.1.1",
            src_port=random.randint(1024, 65535),
            dst_port=443,
            protocol='TCP',
            size=128,
            flags={'RST': 1, 'HTTP2': True}
        )

    def _gen_syn_flood(self):
        return NetworkPacket(
            src_ip=random.choice(self.botnet_ips),
            dst_ip="192.168.1.1",
            src_port=random.randint(1024, 65535),
            dst_port=80,
            protocol='TCP',
            size=64,
            flags={'SYN': 1}
        )

    def _gen_udp_flood(self):
        return NetworkPacket(
            src_ip=random.choice(self.botnet_ips),
            dst_ip="192.168.1.1",
            src_port=random.randint(1024, 65535),
            dst_port=random.randint(1, 65535),
            protocol='UDP',
            size=512
        )

    def _gen_dns_amp(self):
        return NetworkPacket(
            src_ip="192.168.1.1",  # Spoofed target IP
            dst_ip="8.8.8.8",
            src_port=53,
            dst_port=random.randint(1024, 65535),
            protocol='UDP',
            size=4096,
            flags={'DNS': True}
        )

    def _gen_slowloris(self):
        return NetworkPacket(
            src_ip=random.choice(self.botnet_ips),
            dst_ip="192.168.1.1",
            src_port=random.randint(1024, 65535),
            dst_port=80,
            protocol='TCP',
            size=256,
            flags={'ACK': 1, 'Partial': True}
        )
class DDoSDetector(DDoSDetector):
    def _gen_ssl_flood(self):
        return NetworkPacket(
            src_ip=random.choice(self.botnet_ips),
            dst_ip="192.168.1.1",
            src_port=random.randint(1024, 65535),
            dst_port=443,
            protocol='TCP',
            size=2048,
            flags={'SSL': True}
        )

    def generate_traffic(self, ddos_prob=0.25):
        if random.random() < ddos_prob:
            attack_type = random.choice(list(self.attack_generators.keys()))
            self.metrics['attack_types'][attack_type] += 1
            return self.attack_generators[attack_type](), 1
        else:
            return self._gen_normal_traffic(), 0

    def _gen_normal_traffic(self):
        return NetworkPacket(
            src_ip=f"192.168.{random.randint(0,255)}.{random.randint(1,254)}",
            dst_ip=f"10.0.{random.randint(0,255)}.{random.randint(1,254)}",
            src_port=random.randint(1024, 65535),
            dst_port=random.choice([80, 443, 22, 53]),
            protocol=random.choice(['TCP', 'UDP']),
            size=random.randint(64, 1500)
        )

    def _calc_entropy(self, field='src_ip'):
        counts = defaultdict(int)
        for pkt in self.history:
            counts[getattr(pkt, field)] += 1
        probs = np.array(list(counts.values())) / len(self.history)
        return entropy(probs) if len(probs) > 1 else 0

    def extract_features(self, packet):
        self.history.append(packet)
        return [
            int(ipaddress.IPv4Address(packet.src_ip)),
            int(ipaddress.IPv4Address(packet.dst_ip)),
            packet.src_port,
            packet.dst_port,
            {'TCP': 0, 'UDP': 1}.get(packet.protocol, 2),
            packet.size,
            len(packet.flags),
            self._calc_entropy('src_ip'),
            self._calc_entropy('dst_ip'),
            len(self.history)/WINDOW_SIZE
        ]

    def train_models(self, X, y):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
        X_train = self.scaler.fit_transform(X_train)

        self.models['rf'].fit(X_train, y_train)
        self.models['iso'].fit(X_train)
        self.models['autoencoder'].fit(
            X_train, X_train,
            epochs=50,
            batch_size=256,
            validation_split=0.1,
            verbose=0
        )

    def detect(self, packet, true_label):
        features = self.extract_features(packet)
        scaled = self.scaler.transform([features])

        rf_prob = self.models['rf'].predict_proba(scaled)[0][1]
        iso_score = self.models['iso'].decision_function(scaled)[0]
        recon = self.models['autoencoder'].predict(scaled)
        ae_error = np.mean(np.square(scaled - recon))

        anomaly_score = (rf_prob + (1 - iso_score) + ae_error) / 3
        is_attack = anomaly_score > 0.65

        self.metrics['timestamps'].append(time.time())
        self.metrics['entropy_src'].append(features[-3])
        self.metrics['entropy_dst'].append(features[-2])
        self.metrics['packet_rates'].append(features[-1])
        self.metrics['sizes'].append(packet.size)
        self.metrics['true_labels'].append(true_label)
        self.metrics['predictions'].append(int(is_attack))
        self.metrics['anomaly_scores'].append(anomaly_score)

        return is_attack, anomaly_score

    def mitigation_action(self, packet):
        if self.metrics['predictions'][-1]:
            self.metrics['blocked_ips'].add(packet.src_ip)
            self.metrics['actions'].append(2)
        else:
            self.metrics['actions'].append(0)

    def run_simulation(self):
        start = time.time()
        while time.time() - start < SIMULATION_DURATION:
            packet, label = self.generate_traffic()
            is_attack, score = self.detect(packet, label)
            self.mitigation_action(packet)
            time.sleep(PACKET_INTERVAL)

    def plot_results(self):
        plt.figure(figsize=(20, 15))

        # Entropy and Packet Rate Timeline
        plt.subplot(3, 2, 1)
        plt.plot(self.metrics['timestamps'], self.metrics['entropy_src'],
                label='Source Entropy')
        plt.plot(self.metrics['timestamps'], self.metrics['entropy_dst'],
                label='Destination Entropy')
        plt.title('Network Entropy Trends')
        plt.legend()

        # Attack Type Distribution
        plt.subplot(3, 2, 2)
        plt.barh(list(self.metrics['attack_types'].keys()),
                list(self.metrics['attack_types'].values()))
        plt.title('DDoS Attack Type Distribution')

        # Anomaly Score Distribution
        plt.subplot(3, 2, 3)
        sns.kdeplot([s for s, l in zip(self.metrics['anomaly_scores'],
                    self.metrics['true_labels']) if l == 0],
                   label='Normal')
        sns.kdeplot([s for s, l in zip(self.metrics['anomaly_scores'],
                    self.metrics['true_labels']) if l == 1],
                   label='Attack')
        plt.title('Anomaly Score Distribution')

        # Mitigation Actions
        plt.subplot(3, 2, 4)
        action_counts = pd.Series(self.metrics['actions']).value_counts()
        action_counts.index = ['No Action', 'Rate Limit', 'Block']
        action_counts.plot.pie(autopct='%1.1f%%')
        plt.title('Mitigation Actions Taken')

        # ROC Curve
        plt.subplot(3, 2, 5)
        fpr, tpr, _ = roc_curve(self.metrics['true_labels'],
                               self.metrics['anomaly_scores'])
        plt.plot(fpr, tpr, label=f'AUC = {auc(fpr, tpr):.2f}')
        plt.plot([0, 1], [0, 1], 'k--')
        plt.title('ROC Curve')
        plt.legend()

        # Blocked IPs
        plt.subplot(3, 2, 6)
        plt.barh(list(self.metrics['blocked_ips'])[:10], [1]*10)
        plt.title('Top 10 Blocked IP Addresses')

        plt.tight_layout()
        plt.show()

        # Confusion Matrix
        plt.figure(figsize=(8,6))
        cm = confusion_matrix(self.metrics['true_labels'],
                             self.metrics['predictions'])
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                   xticklabels=['Normal', 'Attack'],
                   yticklabels=['Normal', 'Attack'])
        plt.title('Confusion Matrix')
        plt.show()

if __name__ == "__main__":
    detector = DDoSDetector()

    # Generate training data
    print("Training model...")
    training_data = [detector.generate_traffic(ddos_prob=0.5) for _ in range(5000)]
    X = [detector.extract_features(pkt) for pkt, _ in training_data]
    y = [label for _, label in training_data]
    detector.train_models(np.array(X), np.array(y))

    # Run simulation
    print("Starting simulation...")
    detector.run_simulation()

    # Show results
    print("Analyzing results...")
    detector.plot_results()

Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 4, in <module>
    from pip._internal.cli.main import main
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/main.py", line 11, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_runnable_pip
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/build_env.py", line 19, in <module>
    from pip._internal.cli.spinners import open_spinner
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/spinners.py", line 9, in <module>
    from pip._internal.utils.logging import get_indentation
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/util